In [1]:
library(lubridate)
library(worldmet)


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




data <- read.csv("../data/csv/dataMto/080110-99999.csv")
data$date <- ymd_hms(data$date)
head(data)

In [2]:
## Get Data from worlmet
dataMto <- importNOAA(code = c("076000-99999", "080110-99999"),
                      year = 2010:2021,
                      hourly = TRUE,
                      n.cores = 10,
                      quiet = FALSE,
                      path = NA
                     )
#dataMto <- mutate_if(dataMto, is.character, as.factor)

In [3]:
min(dataMto$date)
data <- dataMto

[1] "2010-01-01 UTC"

In [4]:
head(data)

code,station,date,latitude,longitude,elev,ws,wd,air_temp,atmos_pres,⋯,cl_2,cl_3,cl,cl_1_height,cl_2_height,cl_3_height,precip_12,precip,precip_6,pwc
<fct>,<fct>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
076000-99999,"SOCOA, FR",2010-01-01 00:00:00,43.4,-1.683333,25,9.3,240,9.0,997.6,⋯,6,NA,6,1080,1500,NA,NA,NA,NA,NA
076000-99999,"SOCOA, FR",2010-01-01 01:00:00,43.4,-1.683333,25,10.3,240,9.6,998.1,⋯,NA,NA,NA,NA,NA,NA,NA,0,NA,NA
076000-99999,"SOCOA, FR",2010-01-01 02:00:00,43.4,-1.683333,25,8.7,240,9.1,998.3,⋯,NA,NA,NA,NA,NA,NA,NA,0,NA,NA
076000-99999,"SOCOA, FR",2010-01-01 03:00:00,43.4,-1.683333,25,9.3,240,8.9,998.8,⋯,6,NA,6,1080,1500,NA,NA,0,NA,NA
076000-99999,"SOCOA, FR",2010-01-01 04:00:00,43.4,-1.683333,25,8.7,230,8.4,998.9,⋯,NA,NA,NA,NA,NA,NA,NA,0,NA,NA
076000-99999,"SOCOA, FR",2010-01-01 05:00:00,43.4,-1.683333,25,9.3,230,8.8,999.4,⋯,NA,NA,NA,NA,NA,NA,NA,0,NA,NA


In [5]:
all.resumen <- data.frame(matrix(rep(NA, 22), nrow=1))
names(all.resumen) <- c('site', 'start_dt', 'end_dt',
                        'ws', 'wd', 'air_temp', 'atmos_pres', 'visibility', 'dew_point', 'RH', 
                        'ceil_hgt', 'cl_1', 'cl_2', 'cl_3', 'cl', 'cl_1_height', 'cl_2_height', 
                        'cl_3_height', 'precip_12', 'precip_6', 'precip', 'pwc')

```R
for (st in levels(data$code)[1:1]) {

    data.st <- data[data$code == st, ]
    resume <- data.frame(site = rep(st, rep(2)),
                         start_dt = min(data.st$date),
                         end_dt = max(data.st$date)
                        )
    resume <- cbind(resume, data.frame(matrix(rep(NA, 19), nrow=1)))
    
    row.names(resume) <- c("resolucion/h.", "proporcion")
    names(resume) <- c('site', 'start_dt', 'end_dt',
                            'ws', 'wd', 'air_temp', 'atmos_pres', 'visibility', 'dew_point', 'RH', 
                            'ceil_hgt', 'cl_1', 'cl_2', 'cl_3', 'cl', 'cl_1_height', 'cl_2_height', 
                            'cl_3_height', 'precip_12', 'precip_6', 'precip', 'pwc')

    for (cl in 7:ncol(data)) {
        a <- data[data$code == st, c(3, cl)]
        a <- a[complete.cases(a), ]

        if (nrow(a) > 0) {
            minResolut <- max(a$date) - min(a$date)

            for (i in 2:nrow(a)) {
                resolut <- a$date[i] - a$date[i-1]
                if (resolut > 0) {
                    minResolut <- min(minResolut, resolut)
                }
            }

            resume[,names(data)[cl]] <- c(as.numeric(minResolut / 3600),
                               as.numeric(minResolut / 3600)*nrow(a) / nrow(data)
                              )
        }
    }
    all.resumen <- rbind(all.resumen, resume)
}
```

In [14]:
func <- function(st) {

    data.st <- data[data$code == st, ]
    resume <- data.frame(site = rep(st, rep(2)),
                         start_dt = min(data.st$date),
                         end_dt = max(data.st$date)
                        )
    resume <- cbind(resume, data.frame(matrix(rep(NA, 38), nrow=2)))
    
    row.names(resume) <- c("resolucion/h.", "proporcion")
    names(resume) <- c('site', 'start_dt', 'end_dt',
                            'ws', 'wd', 'air_temp', 'atmos_pres', 'visibility', 'dew_point', 'RH', 
                            'ceil_hgt', 'cl_1', 'cl_2', 'cl_3', 'cl', 'cl_1_height', 'cl_2_height', 
                            'cl_3_height', 'precip_12', 'precip_6', 'precip', 'pwc')

    for (cl in 7:ncol(data)) {
        a <- data[data$code == st, c(3, cl)]
        a <- a[complete.cases(a), ]

        if (nrow(a) > 0) {
            minResolut <- max(a$date) - min(a$date)

            for (i in 2:nrow(a)) {
                resolut <- a$date[i] - a$date[i-1]
                if (resolut > 0) {
                    minResolut <- min(minResolut, resolut)
                }
            }

            resume[,names(data)[cl]] <- c(as.numeric(minResolut / 3600),
                               as.numeric(minResolut / 3600)*nrow(a) / nrow(data)
                              )
        }
    }
    resume
}

In [15]:
a

,site,start_dt,end_dt,ws,wd,air_temp,atmos_pres,visibility,dew_point,RH,⋯,cl_2,cl_3,cl,cl_1_height,cl_2_height,cl_3_height,precip_12,precip_6,precip,pwc
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
resolucion/h.,076000-99999,1262304000,1611867600,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.00000000,1.000000000,1.0000000,1.0000000,1.00000000,1.000000000,NA,NA,1.00000000,NA
proporcion,076000-99999,1262304000,1611867600,0.4854042,0.4791467,0.4859141,0.4858575,0.1152158,0.4855124,0.4855072,⋯,0.03324475,0.001740779,0.1067952,0.1036175,0.03324475,0.001740779,NA,NA,0.48269007,NA
11,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
resolucion/h.1,080110-99999,1262322000,1611867600,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.00000000,1.000000000,1.0000000,1.0000000,1.00000000,1.000000000,12.00000000,6.00000000,1.00000000,1.00000000
proporcion1,080110-99999,1262322000,1611867600,0.4311568,0.3896717,0.4308839,0.2535459,0.2847666,0.4308633,0.4308633,⋯,0.20440757,0.051080004,0.3499892,0.3512922,0.20342902,0.051069703,0.07150582,0.03596922,0.06706117,0.02837778


In [16]:
a <- do.call("rbind", lapply(levels(data$code)[1:2], func))

In [17]:
a

,site,start_dt,end_dt,ws,wd,air_temp,atmos_pres,visibility,dew_point,RH,⋯,cl_2,cl_3,cl,cl_1_height,cl_2_height,cl_3_height,precip_12,precip_6,precip,pwc
,<fct>,<dttm>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
resolucion/h.,076000-99999,2010-01-01 00:00:00,2021-01-28 21:00:00,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.00000000,1.000000000,1.0000000,1.0000000,1.00000000,1.000000000,NA,NA,1.00000000,NA
proporcion,076000-99999,2010-01-01 00:00:00,2021-01-28 21:00:00,0.4854042,0.4791467,0.4859141,0.4858575,0.1152158,0.4855124,0.4855072,⋯,0.03324475,0.001740779,0.1067952,0.1036175,0.03324475,0.001740779,NA,NA,0.48269007,NA
resolucion/h.1,080110-99999,2010-01-01 05:00:00,2021-01-28 21:00:00,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.00000000,1.000000000,1.0000000,1.0000000,1.00000000,1.000000000,12.00000000,6.00000000,1.00000000,1.00000000
proporcion1,080110-99999,2010-01-01 05:00:00,2021-01-28 21:00:00,0.4311568,0.3896717,0.4308839,0.2535459,0.2847666,0.4308633,0.4308633,⋯,0.20440757,0.051080004,0.3499892,0.3512922,0.20342902,0.051069703,0.07150582,0.03596922,0.06706117,0.02837778
